In [13]:
# Imports

import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
classifiers = ["EDC", "AMAXSC", "M4GP", "lda",  "decision_tree", "MLP", "random_forest", "svm_rbf"]
score_metric = "auc_score"
score_metric_index = 0 if score_metric == "auc_score" else 1

result_files = os.listdir("results")
results = []
for result_file in result_files:
    dataset, search_strategy, optimiser, random_seed = result_file.split(
        "."
    )[0].split("-")
    with open(f"results/{result_file}", "rb") as f:
        res = pickle.load(f)
    for elapsed_time, auc_score, accuracy_score in zip(res["elapsed_time"], res["auc_score"], res["accuracy_score"]):
        results.append(
            [
                dataset,
                "EDC",  # "classifier
                search_strategy,
                optimiser,
                elapsed_time,
                auc_score,
                accuracy_score,
                random_seed,
            ]
        )

result_files = os.listdir("comparison_results")
for result_file in result_files:
    dataset, classifier, random_seed = result_file.split(
        "."
    )[0].split("-")
    with open(f"comparison_results/{result_file}", "rb") as f:
        res = pickle.load(f)
    for elapsed_time, auc_score, accuracy_score in zip(res["elapsed_time"], res["auc_score"], res["accuracy_score"]):
        results.append(
            [
                dataset,
                classifier,  # "classifier
                None,
                None,
                elapsed_time,
                auc_score,
                accuracy_score,
                random_seed,
            ]
        )


df = pd.DataFrame(
    columns=[
        "dataset",
        "classifier",
        "search_strategy",
        "optimiser",
        "elapsed_time",
        "auc_score",
        "accuracy_score",
        "random_seed"
    ],
    data=results,
)

In [15]:
print(df)

       dataset classifier search_strategy     optimiser  elapsed_time  \
0     BANKNOTE        EDC            beam  hill_climber      1.203219   
1       BREAST        EDC            beam  hill_climber     38.952624   
2     DIABETES        EDC            beam  hill_climber      2.500993   
3       CREDIT        EDC            beam  hill_climber     42.151524   
4    OCCUPANCY        EDC            beam  hill_climber      2.051289   
..         ...        ...             ...           ...           ...   
858  OCCUPANCY       M4GP            None          None   9687.420261   
859  OCCUPANCY       M4GP            None          None   7649.649928   
860  OCCUPANCY       M4GP            None          None   9140.788291   
861  OCCUPANCY       M4GP            None          None   9044.762067   
862      ADULT       M4GP            None          None  69212.286039   

     auc_score  accuracy_score random_seed  
0     1.000000        0.992754   1805819_0  
1     0.605556        0.689655   

In [16]:
dataset_grouped = df.groupby(["dataset"])
all_scores = []
all_stds = []
for dataset_group in dataset_grouped:
    scores = {}
    stds = {}
    dataset = dataset_group[0][0]
    if dataset == "BANANA" or (dataset.startswith("AD0")) or dataset == "HEPATITIS" or dataset == "WISCONSIN":
        continue
    for classifier_group in dataset_group[1].groupby(["classifier"]):
        number_of_folds_done = len(classifier_group[1])

        classifier = classifier_group[0][0]
        mean_auc = classifier_group[1]["auc_score"].mean()
        std_auc = classifier_group[1]["auc_score"].std()
        mean_acc = classifier_group[1]["accuracy_score"].mean()
        std_acc = classifier_group[1]["accuracy_score"].std()
        scores[classifier] = (mean_auc, mean_acc)
        stds[classifier] = (std_auc, std_acc)
        


    # Get the ranks
    for classifier in classifiers:
        if classifier not in scores:
            scores[classifier] = (0, 0)
            stds[classifier] = (0, 0)

    for classifier in classifiers:
        current_score = scores[classifier][score_metric_index]
        rank = 1
        for other_classifier in classifiers:
            if other_classifier == classifier:
                continue
            if scores[other_classifier][score_metric_index] > current_score:
                rank += 1
        scores[classifier] = (scores[classifier][0], scores[classifier][1], rank)
    

    print(f"{dataset}", end="&")
    if score_metric == "auc_score":
        # Print AUC scores
        for clf in classifiers:
            print(f"{scores[clf][0]:.4f} ($\\pm{stds[clf][0]:.2f}$)", end="&")

    if score_metric == "accuracy_score":
        # Print Acc
        for clf in classifiers:
            print(f"{scores[clf][1]:.4f} ($\\pm{stds[clf][1]:.2f}$)", end="&")
    
    # # Print rank
    # for clf in classifiers:
    #     print(f"{scores[clf][2]}", end="&")
    
    print(f"\b\\\\")

    all_scores.append(scores)
    all_stds.append(stds)
    # print(f"{dataset} & {scores['EDC'][0]:.4f} & {scores['random_forest'][0]:.4f} & {scores['EDC'][1]:.4f} & {scores['random_forest'][1]:.4f} \\\\")



# Get average ranks
average_ranks = {}
average_aucs = {}
average_accs = {}
for classifier in classifiers:
    average_ranks[classifier] = 0
    average_aucs[classifier] = 0
    average_accs[classifier] = 0
    for dataset in all_scores:
        if classifier not in dataset:
            continue
        average_ranks[classifier] += dataset[classifier][2]
        average_aucs[classifier] += dataset[classifier][0]
        average_accs[classifier] += dataset[classifier][1]
    average_ranks[classifier] /= len(all_scores)
    average_aucs[classifier] /= len(all_scores)
    average_accs[classifier] /= len(all_scores)

print("\\midrule")
print(f"Average Score&", end="")
for classifier in classifiers:
    print(f"{average_aucs[classifier]:.4f}", end="&")
print(f"\b\\\\")

print(f"Average Rank&", end="")
for classifier in classifiers:
    print(f"{average_ranks[classifier]:.2f}", end="&")
print(f"\b\\\\")

ADULT&0.8889 ($\pm0.00$)&0.8070 ($\pm0.01$)&0.7696 ($\pmnan$)&0.9017 ($\pm0.00$)&0.7301 ($\pm0.01$)&0.9011 ($\pm0.01$)&0.8799 ($\pm0.00$)&0.8980 ($\pm0.00$)\\
BANKNOTE&1.0000 ($\pm0.00$)&0.9815 ($\pm0.02$)&0.9994 ($\pm0.00$)&0.9996 ($\pm0.00$)&0.9790 ($\pm0.01$)&1.0000 ($\pm0.00$)&0.9999 ($\pm0.00$)&1.0000 ($\pm0.00$)\\
BREAST&0.6697 ($\pm0.11$)&0.6169 ($\pm0.11$)&0.6140 ($\pm0.09$)&0.6359 ($\pm0.14$)&0.5896 ($\pm0.09$)&0.7006 ($\pm0.16$)&0.6828 ($\pm0.15$)&0.7088 ($\pm0.10$)\\
CREDIT&0.9177 ($\pm0.03$)&0.8962 ($\pm0.03$)&0.8686 ($\pm0.03$)&0.9243 ($\pm0.05$)&0.8116 ($\pm0.04$)&0.9101 ($\pm0.05$)&0.9350 ($\pm0.02$)&0.9196 ($\pm0.03$)\\
CYLINDER&0.7346 ($\pm0.09$)&0.5471 ($\pm0.05$)&0.7027 ($\pm0.09$)&0.7775 ($\pm0.07$)&0.5944 ($\pm0.09$)&0.8443 ($\pm0.07$)&0.8697 ($\pm0.04$)&0.7590 ($\pm0.10$)\\
DIABETES&0.8303 ($\pm0.04$)&0.7985 ($\pm0.06$)&0.7240 ($\pm0.05$)&0.8289 ($\pm0.03$)&0.6733 ($\pm0.04$)&0.8425 ($\pm0.02$)&0.8261 ($\pm0.07$)&0.8356 ($\pm0.05$)\\
IONOSPHERE&0.8944 ($\pm0.05$)&